# 작동 확인

In [1]:
import mysql.connector

remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cur = remote.cursor()

remote.close()

### 시간별 날씨 (df_h_all)

In [16]:
import pandas as pd

df_h_all = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Weather/hourly_everything_2024.csv", 
                 encoding="cp949", usecols=[2,3,5,7,11,19,21,23])

df_h_all.fillna(0, inplace=True)
df_h_all.head()

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),적설(cm)
0,2024-01-01 00:00,0.6,0.0,1.9,97,0.0,0.0,2.4
1,2024-01-01 01:00,0.5,0.0,2.2,97,0.0,0.0,2.4
2,2024-01-01 02:00,0.4,0.0,0.4,98,0.0,0.0,2.4
3,2024-01-01 03:00,-0.1,0.0,1.7,98,0.0,0.0,2.4
4,2024-01-01 04:00,-0.2,0.0,2.2,98,0.0,0.0,2.4


### 일시/ 시간 따로 분리

In [25]:

dates = []
times = []

for idx, row in df_h_all.iterrows():

    date = row['일시'].split(' ')[0]
    time = row['일시'].split(' ')[1]
    time = int(time.split(':')[0])

    dates.append(date)
    times.append(time)

df_h_all['일시'] = dates
df_h_all['시간'] = times

In [26]:
df_h_all

,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),적설(cm),시간
0,2024-01-01,0.6,0.0,1.9,97,0.0,0.0,2.4,0
1,2024-01-01,0.5,0.0,2.2,97,0.0,0.0,2.4,1
2,2024-01-01,0.4,0.0,0.4,98,0.0,0.0,2.4,2
3,2024-01-01,-0.1,0.0,1.7,98,0.0,0.0,2.4,3
4,2024-01-01,-0.2,0.0,2.2,98,0.0,0.0,2.4,4
...,...,...,...,...,...,...,...,...,...
8779,2024-12-31,0.5,0.0,1.9,46,0.0,0.0,0.0,19
8780,2024-12-31,0.0,0.0,2.4,48,0.0,0.0,0.0,20
8781,2024-12-31,-0.7,0.0,1.0,51,0.0,0.0,0.0,21
8782,2024-12-31,-1.0,0.0,0.7,53,0.0,0.0,0.0,22


# CSV로 df_d_all 저장 (utf-8)

In [27]:
df_h_all.to_csv("/home/dan/dev_ws/EDA/project/data/Weather/hourly_everything_2024B.csv", 
                sep=",", index=False, encoding="utf-8")

# DB Table (weather_daily) 만들기

In [33]:
remote.close()

In [35]:
remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)

cursor.execute("CREATE TABLE weather_hourly (weather_date DATE, \
               weather_hour TINYINT,\
               temp FLOAT, rain FLOAT, wind FLOAT, humid FLOAT,\
               sunlight_direct FLOAT, radiation_total FLOAT, snow FLOAT)")

remote.commit()
remote.close()

# DB 에 집어넣기

In [37]:
import mysql.connector

remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)

sql= """INSERT INTO weather_hourly VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for i,row in df_h_all.iterrows(): 

    weather_date = row['일시']
    weather_hour = row['시간']
    temp = row['기온(°C)']
    rain = row['강수량(mm)']
    wind = row['풍속(m/s)']
    humid = row['습도(%)']
    sunlight_direct = row['일조(hr)']
    radiation_total = row['일사(MJ/m2)']
    snow = row['적설(cm)']

    content = (weather_date, weather_hour, temp, rain, humid, wind,
               sunlight_direct, radiation_total, snow)

    cursor.execute(sql, content)

    print(content)
    

remote.commit()
remote.close()

('2024-01-01', 0, 0.6, 0.0, 97, 1.9, 0.0, 0.0, 2.4)
('2024-01-01', 1, 0.5, 0.0, 97, 2.2, 0.0, 0.0, 2.4)
('2024-01-01', 2, 0.4, 0.0, 98, 0.4, 0.0, 0.0, 2.4)
('2024-01-01', 3, -0.1, 0.0, 98, 1.7, 0.0, 0.0, 2.4)
('2024-01-01', 4, -0.2, 0.0, 98, 2.2, 0.0, 0.0, 2.4)
('2024-01-01', 5, 0.3, 0.0, 99, 1.2, 0.0, 0.0, 2.4)
('2024-01-01', 6, 0.5, 0.0, 99, 2.0, 0.0, 0.0, 2.4)
('2024-01-01', 7, 0.4, 0.0, 98, 2.1, 0.0, 0.0, 2.4)
('2024-01-01', 8, 0.2, 0.0, 98, 0.8, 0.0, 0.0, 2.4)
('2024-01-01', 9, 0.6, 0.0, 99, 1.3, 0.5, 0.27, 2.4)
('2024-01-01', 10, 2.4, 0.0, 95, 1.4, 1.0, 0.82, 2.4)
('2024-01-01', 11, 4.4, 0.0, 82, 2.0, 1.0, 1.36, 2.2)
('2024-01-01', 12, 5.7, 0.0, 73, 2.3, 1.0, 1.7, 1.7)
('2024-01-01', 13, 7.3, 0.0, 70, 1.6, 0.8, 1.87, 0.9)
('2024-01-01', 14, 6.5, 0.0, 68, 2.1, 0.0, 0.65, 0.5)
('2024-01-01', 15, 6.4, 0.0, 68, 1.2, 0.0, 0.5, 0.2)
('2024-01-01', 16, 5.8, 0.0, 68, 2.2, 0.0, 0.18, 0.0)
('2024-01-01', 17, 5.5, 0.0, 71, 2.3, 0.0, 0.08, 0.0)
('2024-01-01', 18, 4.9, 0.0, 71, 1.1, 0.0, 0.01